In [ ]:
# 1. GTFS Static Data
"""
Description: GTFS (General Transit Feed Specification) static data 
includes scheduled transit services, stops, routes, and times. 
It's suitable for analyzing scheduled transit times.
"""
import pandas as pd

# Load GTFS files
routes_df = pd.read_csv('routes.txt')
trips_df = pd.read_csv('trips.txt')
stop_times_df = pd.read_csv('stop_times.txt')
stops_df = pd.read_csv('stops.txt')

# Filter routes crossing the Bay Bridge
bay_bridge_routes = routes_df[routes_df['route_long_name'].str.contains('Bay Bridge')]

# Merge data to find stop times for specific routes
trips_with_times = trips_df.merge(stop_times_df, on='trip_id')
bay_bridge_trips = trips_with_times[trips_with_times['route_id'].isin(bay_bridge_routes['route_id'])]

# Calculate transit times between stops
oakland_stop_id = stops_df[stops_df['stop_name'] == 'Oakland'].iloc[0]['stop_id']
sf_stop_id = stops_df[stops_df['stop_name'] == 'San Francisco'].iloc[0]['stop_id']
bay_bridge_trips = bay_bridge_trips[(bay_bridge_trips['stop_id'] == oakland_stop_id) | (bay_bridge_trips['stop_id'] == sf_stop_id)]

# Calculate time differences
bay_bridge_trips['arrival_time'] = pd.to_datetime(bay_bridge_trips['arrival_time'])
bay_bridge_trips['transit_time'] = bay_bridge_trips.groupby('trip_id')['arrival_time'].diff().dt.total_seconds() / 60

print(bay_bridge_trips[['trip_id', 'arrival_time', 'transit_time']])

In [ ]:
# 2. GTFS-RT (Real-Time) Data
"""
Description: GTFS-RT provides real-time updates about vehicle positions, trip updates, and service alerts.
"""
import requests
from google.transit import gtfs_realtime_pb2

# Fetch GTFS-RT data
response = requests.get('URL_TO_GTFS_RT_FEED')
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

# Extract vehicle positions
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle
        print(f"Vehicle ID: {vehicle.vehicle.id}, Position: {vehicle.position.latitude}, {vehicle.position.longitude}, Timestamp: {vehicle.timestamp}")


In [ ]:
# 3. Transit Trip Planner API
"""
Description: This API provides programmatic access to transit directions, 
travel times, and route information between specified locations.
"""
import requests

# API credentials
api_key = 'YOUR_API_KEY'

# Request transit directions
response = requests.get(f'https://api.511.org/transit/tripplanner?origin=Oakland&destination=San+Francisco&key={api_key}')
trip_data = response.json()

# Extract transit time
for itinerary in trip_data['Itineraries']:
    print(f"Duration: {itinerary['Duration']}, Start Time: {itinerary['StartTime']}, End Time: {itinerary['EndTime']}")


In [ ]:
# 4. Transit Real-Time API
"""
Description: Provides real-time information on vehicle positions, estimated arrival times, and trip statuses.
"""
import requests

# API credentials
api_key = 'YOUR_API_KEY'

# Request real-time vehicle positions
response = requests.get(f'https://api.511.org/transit/vehiclepositions?operator_id=AC_TRANSIT&key={api_key}')
real_time_data = response.json()

# Extract vehicle locations
for vehicle in real_time_data['Vehicles']:
    print(f"Vehicle ID: {vehicle['VehicleId']}, Current Position: {vehicle['CurrentPosition']}, Timestamp: {vehicle['Timestamp']}")


In [ ]:
# 5. Traffic Data API (Optional)
"""
Description: Provides real-time traffic information, which can be useful for buses or other vehicles affected by traffic on the Bay Bridge.
"""
import requests

# API credentials
api_key = 'YOUR_API_KEY'

# Request traffic data
response = requests.get(f'https://api.511.org/traffic?origin=Oakland&destination=San+Francisco&key={api_key}')
traffic_data = response.json()

# Extract traffic conditions
for route in traffic_data['Routes']:
    print(f"Route: {route['Name']}, Traffic Speed: {route['TrafficSpeed']}, Travel Time: {route['TravelTime']}")


In [ ]:
# Calculating Transit Time

import requests
import pandas as pd
from datetime import datetime

# API credentials
api_key = 'YOUR_API_KEY'

# Request real-time trip updates
response = requests.get(f'https://api.511.org/transit/tripupdates?operator_id=AC_TRANSIT&key={api_key}')
trip_updates = response.json()

# Load GTFS static data
routes_df = pd.read_csv('routes.txt')
trips_df = pd.read_csv('trips.txt')
stop_times_df = pd.read_csv('stop_times.txt')
stops_df = pd.read_csv('stops.txt')

# Filter for
